In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = preprocess_input(x_train.astype("float32"))
x_test = preprocess_input(x_test.astype("float32"))

# Load VGG16 without top classifier
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

# Freeze **most** layers – keep last block trainable
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom classifier
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train only unfreezed layers + new layers
history = model.fit(x_train, y_train, epochs=1, batch_size=64,
                    validation_data=(x_test, y_test))

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", acc)

# Prediction example
pred = model.predict(x_test[:1])
print("Predicted class:", np.argmax(pred))


782/782 ━━━━━━━━━━━━━━━━━━━━ 683s 867ms/step - accuracy: 0.1015 - loss: 2.3956 - val_accuracy: 0.1000 - val_loss: 2.3026
313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 199ms/step - accuracy: 0.1000 - loss: 2.3026
Test Accuracy: 0.10000000149011612
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
Predicted class: 3
